# Minishell

Notes on the various stages of minishell implementation.

## Parser
---

To implement prompt parsing, we will use a finite state machine, which represents the control flow needed to break down the command into elements. The finite state machine consists of one or more states and a series of transitions between the states, according to which the operations necessary for parsing are performed.

Once the command is divided into elements, we will use a data structure such as a concatenated list to represent the individual elements. This way, it will be possible to manipulate each element separately to perform the operations required by the command. We opted for a concatenated list because it allows us to have a dynamic structure.

Definition of the list:

In [ ]:
typedef struct s_cmd
{
	char			*token;
	char			type;
	struct s_cmd	*next;
	struct s_cmd	*prev;
}	t_cmd;

The prompt is divided into tokens, of the following types:
- STANDARD: It is the token representing commands, options and arguments.
- REDIRECT: It is the token representing redirection characters.
- PIPE: It is the token representing the pipe operator.
- BOOLEAN: It is the token representing Boolean operators. For convenience, even round brackets, outside of quotes, are considered booleans.
- NONE: It is a convention within the parsing system to indicate that the token has not yet been defined.

``` c
prompt = cat < file 

        +------------------------+    +------------------------+    +------------------------+                                              
        |   addr: 0x7fffea48     |    |   addr: 0x2f96a8d1     |    |   addr: 0x1ef1bd7d     |                                             
        +------------------------+    +------------------------+    +------------------------+                                               
head -> |   token = "cat "       | -> |   token = "<"          | -> |   token = "file"       | -> NULL  
        |   type  = STANDARD     |    |   type  = REDIRECT     |    |   type  = STANDARD     |                                      
        |   next  = 0x2f96a8d1   |    |   next  = 0x1ef1bd7d   |    |   next  = NULL         |                                             
        |   prev  = NULL         |    |   prev  = 0x7fffea48   |    |   prev  = 0x2f96a8d1   |                                             
        +------------------------+    +------------------------+    +------------------------+
```         

Delimiters between tokens are, if outside single and double quotes:
- Spaces and tabs
- characters of type REDIRECT
- characters of type PIPE
- characters of type BOOLEAN

Some examples of parsing:

``` c
prompt = echo "'$USER'42 $SHELL" '|' > file | (grep "USER" && echo 42)
```
| Token                   | Type     | 
| ----------------------- | ---      |
| echo                    | STANDARD |
| '$USER'42 /usr/bin/bash | STANDARD |
| \|                      | STANDARD |
| >                       | REDIRECT |
| file                    | STANDARD |
| \|                      | PIPE     |
| (                       | BOOLEAN  |
| grep                    | STANDARD |
| USER                    | STANDARD |
| &&                      | BOOLEAN  |
| echo                    | STANDARD |
| 42                      | STANDARD |
| )                       | BOOLEAN  |


## Executor
---

The executor is the main module and is responsible for executing commands. To do this, it relies on the parser module, which provides it with the commands to execute. The executor is responsible for creating the processes needed to execute the commands, and for handling pipes, redirections, and booleans.

```c
                                               +-----------------+            
                                               |   Execution     |            
                                               |   System        |            
                                               +-----------------+            
                                                        |                     
                                                        v                     
                                               +-----------------+            
                                               |     Routing     |            
                                               +-----------------+            
                                                     |  |  |               
                                                     |  |  |               
                                    +----------------+  |  +----------------+  
                                    |                   |                   |  
                                    v                   v                   v  
                           +-----------------+ +-----------------+ +-----------------+
                           |  Redirections   | |      Pipes      | |  Booleans       |
                           +-----------------+ +-----------------+ +-----------------+

                                                     Execute
                           +---------------------------------------------------------+                                              
                                          |                             |
                                          v                             v
                                 +-----------------+           +-----------------+                  
                                 | Built-in        |           | External        |                                 
                                 +-----------------+           +-----------------+                                 
```                      

1. The main execution_system function handles command execution. It takes as input a pointer to a t_cmd structure (the command line previously processed by Parse) and a pointer to a t_var structure (containing the variables, of shell and env).

2. Within execution_system, an args array is initialized to contain the arguments to the current command and an exe string to store the path to the executable to be executed.

3. A loop is then executed that continues as long as there are commands to execute and the type of the current command is not a Boolean type.

4. If the type of the current command is STANDARD, an additional loop is executed to read and store the arguments of the current command in the args array. The first argument (the name of the executable) is also assigned to the string exe.

5. A check is then performed to determine whether the current executable is a built-in command, whether the executable exists in the current path, or whether it is necessary to look up the executable in the environment path. If any of these conditions are met, the exec_router function is called to route the execution.

6. The exec_router function handles execution routing based on the command type. If the current command has redirections, the function calls the redirections function to handle them. If the command also has a pipe, the pipes function is called. If the current executable is a built-in command, the execute_builtin function is called. Otherwise, if the current executable is an external command, the execute function is called.

7. After execution of the current command, resources, such as the args array and the exe string, are freed. Closing the descriptor files and restoring the terminal is also handled.

8. The loop continues with the next command unless certain exit conditions occur, such as receiving an interrupt signal or an execution error.


#### Redirections

Redirections were handled by a cascade mechanism, with the following flow:
- The first redirection is executed, and the current fd and the new redirected fd are saved in the t_fd list.
- It continues to recursively execute redirects, until the last one.
- It goes back into the router and is checked to see if the next token is of type PIPE, in which case the command is executed and handled by pipes.
- Otherwise the command is executed normally, which can be a built-in or an external.
- When finished, all fd are closed, then the last one is closed and the previous one is restored, until the terminal fd is returned.

#### Heredoc

Heredoc was implemented via pipe(), as pipe is a special type of file with the difference of having 2 fd, we used it as a temporary file, so we get the document via readline(), write it to the write side [1] in pipe and redirect the input to the read side of pipe [0].

NOTE: Since in order to get the input with readline() from the terminal and display the output, we need the terminal fds, the terminal is then temporarily reset while running heredoc.

#### Pipe

Pipe is handled in the following way:

- The current input fd is saved.
- A new child process is created.
- In the child process the output on the write side of pipe is redirected if the current output is from the terminal and the command is executed, otherwise it is executed but without writing it to pipe.
- Terminal fd output is restored.
- You redirect the input to the read side of pipe.